In [5]:
!pip install pyspark==3.5.6 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 33.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895856 sha256=86c472be7bbdd71c72337c420b1483e6fd57adf713b0405331664fbdd2c9e003
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.0
    Uninstalling pyspark-4.0.0:
      Successfully uninstalled pyspark-4.0.0

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["KUBECONFIG"] = os.path.join(ROOT_DIR, "k8s-creds", 'config')
os.environ["PYSPARK_SUBMIT_ARGS"] = f"""
--conf spark.executor.memory=1G
--conf spark.executor.cores=1
pyspark-shell
"""

In [4]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("k8s-test-app")
    .master("spark://host.docker.internal:30077")
    .config("spark.kubernetes.container.image", "bitnami/spark:latest")
    .config("spark.kubernetes.namespace", "default")
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
    .config("spark.executor.instances", "2")      # сколько Executors запустить
    .config("spark.executor.cores", "1")          # по одному CPU‐ядру
    .config("spark.executor.memory", "1g")      # по 512 МБ памяти
    .config("spark.driver.memory", "1g")        # драйверу тоже ограничим RAM, если нужно
    .config("spark.driver.host", "host.docker.internal")
    .config("spark.jars.ivy", "/tmp/.ivy2")  # укажи директорию вручную
    .config("spark.driver.bindAddress",  "0.0.0.0")
    .config("spark.driver.port",         "45555")
    .config("spark.blockManager.port",   "45556")
    .getOrCreate())

df = spark.range(1000)
df.selectExpr("sum(id)").show()

spark.stop()


+-------+
|sum(id)|
+-------+
| 499500|
+-------+

